In [31]:
import os
import shutil


In [1]:
from pyspark import SparkContext
sc = SparkContext("local","lab_1_mapreduce")

25/05/21 11:32:38 WARN Utils: Your hostname, mahesh-Aspire-A315-59 resolves to a loopback address: 127.0.1.1; using 192.168.1.42 instead (on interface wlp43s0)
25/05/21 11:32:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 11:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### **1. Word Count with Filtering**
**Input:** A file named `documents.txt`, each line containing a sentence.  
**Task:** Write PySpark code to:
- Count the frequency of each word (case-insensitive).
- Exclude stopwords like `["a", "the", "is", "in", "at", "on", "and"]`.
- Output should be sorted by frequency in descending order.

In [ ]:
docs = sc.textFile("..//lab_datasets/documents.txt")

In [11]:
contents = docs.flatMap(lambda line: line.strip().split("\t")[1].split(" "))\
    .filter(lambda word: word.lower() not in ["a", "the", "is", "in", "at", "on", "and"])\
        .map(lambda word: (word, 1))\
            .reduceByKey(lambda a, b: a + b)\
    

In [14]:
with open("word_count_with_filtering.txt", "w") as f:
    contents.saveAsTextFile("word_count_with_filtering")

### **2. Most Purchased Product**
**Input:** A CSV file `transactions.csv` in the format: `user_id,product_id,timestamp`.  
**Task:** 
- Find the product(s) with the highest number of purchases.
- Save the result as: `("product_id", count)`.

In [ ]:
docs = sc.textFile("../lab_datasets/transactions.csv")
sorted_counts =  docs.map(lambda line: line.strip().split(","))\
    .map(lambda fields: (fields[1], 1))\
        .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda x: x[1], ascending=False)


In [ ]:
# find the highest purchase count
# get the first element
product, count = sorted_counts.take(1)[0]

sorted_counts.filter(lambda x: x[1] == count)\
    .saveAsTextFile('highest_purchase_count')

### **4. Average Rating Per Product**
**Input:** A CSV file `reviews.csv` in the format: `user_id,product_id,rating`.  
**Task:** 
- Compute the **average rating** for each product.
- Output format: `("product_id", average_rating)` rounded to 2 decimal places.

In [ ]:
docs = sc.textFile("../lab_datasets/reviews.csv")
#remove the output dir and files if it already exists

#delete a directory if it already exists including all files in the directory
if os.path.exists("average_rating_per_product"):
    shutil.rmtree("average_rating_per_product")
    
docs.map(lambda line: line.strip().split(","))\
    .map(lambda fields: (fields[1], int(fields[2])))\
        .reduceByKey(lambda a, b: a*0.5 + b*0.5)\
            .sortByKey(ascending=True)\
                .saveAsTextFile('average_rating_per_product')

### **5. Most Active User by Reviews**
**Input:** Same as above (`reviews.csv`).  
**Task:** 
- Identify the user(s) who submitted the **highest number of reviews**.
- Output format: `("user_id", review_count)`.

In [ ]:
docs = sc.textFile("../lab_datasets/reviews.csv")

sorted_user_review_counts = docs.map(lambda line: line.strip().split(","))\
    .map(lambda fields: (fields[0], 1))\
        .reduceByKey(lambda a, b: a + b)\
            .sortBy(lambda x: x[1], ascending=False)   

In [37]:
user, num_reviews = sorted_user_review_counts.take(1)[0]

In [38]:
if os.path.exists("highest_review_count"):
    shutil.rmtree("highest_review_count")
    
sorted_user_review_counts.filter(lambda x: x[1] == num_reviews)\
    .saveAsTextFile('highest_review_count')